In [ ]:
!pip install pandas

In [90]:
import pandas as pd
from collections import Counter
import re
import numpy as np

In [91]:
df = pd.read_csv('treatment.csv')

In [92]:
df

,treatmentid,patientunitstayid,treatmentoffset,treatmentstring,activeupondischarge
0,9579899,242895,838,cardiovascular|arrhythmias|anticoagulant admin...,False
1,8788989,242895,512,cardiovascular|consultations|Cardiology consul...,False
2,10293108,242895,838,cardiovascular|non-operative procedures|extern...,False
3,9017080,242895,70,pulmonary|vascular disorders|VTE prophylaxis|l...,False
4,9853526,242895,70,cardiovascular|consultations|Cardiology consul...,False
...,...,...,...,...,...
38285,82612510,3352230,2176,pulmonary|ventilation and oxygenation|mechanic...,False
38286,82477425,3352230,2176,neurologic|pain / agitation / altered mentatio...,False
38287,82402829,3352231,117,cardiovascular|non-operative procedures|intraa...,False
38288,82335549,3352333,317,gastrointestinal|consultations|Gastroenterolog...,False


In [93]:
treatmentIDList = df['treatmentid'].tolist()
repetitions = Counter(treatmentIDList)


In [94]:
[v for k,v in repetitions.items() if v>=2]

[]

In [95]:
treatmentList = df['treatmentstring'].tolist()
treatmentList2 = []
for treatment in treatmentList:
    for treat in treatment.split('|'):
        treatmentList2.append(treat)

In [96]:
repetitionsTreatment = Counter(treatmentList2)

In [97]:
sorted(repetitionsTreatment.items(), key=lambda x:x[1], reverse = True)

[('cardiovascular', 9606),
 ('pulmonary', 8409),
 ('medications', 7594),
 ('neurologic', 4667),
 ('gastrointestinal', 4474),
 ('ventilation and oxygenation', 3651),
 ('infectious diseases', 3465),
 ('pain / agitation / altered mentation', 2653),
 ('analgesics', 2322),
 ('consultations', 2296),
 ('renal', 2225),
 ('endocrine', 2192),
 ('therapeutic antibacterials', 1786),
 ('vascular disorders', 1773),
 ('myocardial ischemia / infarction', 1715),
 ('VTE prophylaxis', 1588),
 ('surgery', 1515),
 ('intravenous fluid', 1492),
 ('normal saline administration', 1381),
 ('mechanical ventilation', 1381),
 ('glucose metabolism', 1378),
 ('stress ulcer prophylaxis', 1328),
 ('insulin', 1270),
 ('radiologic procedures / bronchoscopy', 1194),
 ('electrolyte correction', 1190),
 ('shock', 1148),
 ('bronchodilator', 1099),
 ('arrhythmias', 1052),
 ('cultures / immuno-assays', 1036),
 ('cultures', 1019),
 ('sedative agent', 910),
 ('vasopressors', 896),
 ('non-operative procedures', 818),
 ('electrol

In [98]:
dfTreatment = df[df['treatmentstring'].str.contains("cardiovascular")]
dfTreatment = dfTreatment[['patientunitstayid','treatmentstring','activeupondischarge']]
dfTreatment

,patientunitstayid,treatmentstring,activeupondischarge
0,242895,cardiovascular|arrhythmias|anticoagulant admin...,False
1,242895,cardiovascular|consultations|Cardiology consul...,False
2,242895,cardiovascular|non-operative procedures|extern...,False
4,242895,cardiovascular|consultations|Cardiology consul...,False
6,242895,cardiovascular|consultations|Cardiology consul...,False
...,...,...,...
38276,3352230,cardiovascular|myocardial ischemia / infarctio...,False
38278,3352230,cardiovascular|shock|vasopressors|epinephrine ...,False
38281,3352230,cardiovascular|shock|inotropic agent|dobutamine,False
38283,3352230,cardiovascular|non-operative procedures|intraa...,False


In [99]:
listPatients = dfTreatment['patientunitstayid'].tolist()
repetitionsPatient = Counter(listPatients)
repetitionsPatient

Counter({242895: 17,
         243097: 16,
         243999: 3,
         244477: 3,
         244707: 5,
         246997: 3,
         247261: 3,
         249328: 37,
         249805: 15,
         250073: 1,
         250574: 3,
         250719: 12,
         250724: 30,
         251111: 6,
         257287: 14,
         257802: 7,
         258915: 20,
         260468: 21,
         260647: 10,
         260860: 3,
         261520: 22,
         262775: 3,
         263279: 5,
         263850: 54,
         264276: 42,
         264423: 3,
         264901: 21,
         267196: 1,
         267829: 2,
         272927: 1,
         275329: 6,
         276562: 6,
         276594: 2,
         276815: 9,
         280350: 18,
         281132: 14,
         282439: 6,
         284265: 22,
         284517: 1,
         287848: 3,
         288133: 8,
         289502: 2,
         291628: 3,
         292154: 5,
         294032: 14,
         295112: 1,
         295916: 18,
         296926: 4,
         296927: 13,


In [100]:
#Treatment: treatmentstring (obj)
#Patient: gender, age, apacheadmissiondx, admissionweight, admissionheight, ethnicity


In [101]:
dfDiagnosis = pd.read_csv('diagnosis.csv', na_values="nan")
dfDiagnosis = dfDiagnosis.dropna()

In [102]:
#dfDiagnosis = dfDiagnosis[dfDiagnosis['diagnosisstring'].str.contains("cardiovascular")]
dfDiagnosis

,diagnosisid,patientunitstayid,activeupondischarge,diagnosisoffset,diagnosisstring,icd9code,diagnosispriority
0,7607199,346380,False,5028,cardiovascular|ventricular disorders|hypertension,"401.9, I10",Other
1,7570429,346380,False,685,neurologic|altered mental status / pain|change...,"780.09, R41.82",Major
2,7705483,346380,True,5035,cardiovascular|shock / hypotension|hypotension,"458.9, I95.9",Major
3,7848601,346380,True,5035,neurologic|altered mental status / pain|schizo...,"295.90, F20.9",Major
4,7451475,346380,False,5028,pulmonary|disorders of vasculature|pulmonary e...,"415.19, I26.99",Major
...,...,...,...,...,...,...,...
24973,43897237,3158501,True,11,neurologic|altered mental status / pain|enceph...,"348.31, G93.41",Other
24974,44151864,3158501,True,11,hematology|white blood cell disorders|leukocyt...,"288.8, D72.829",Other
24975,44379558,3158501,True,11,pulmonary|disorders of acid base|respiratory a...,"276.2, E87.2",Other
24976,44000639,3158501,True,11,neurologic|altered mental status / pain|delirium,"293.0, F05",Other


In [103]:
listICDcodes = dfDiagnosis['icd9code'].tolist()
validIndices = []
i=0
for icd in listICDcodes:
    icdCode1 = icd.split(',')[0]
    if re.match(r'^-?\d+(?:\.\d+)$', icdCode1) is not None and float(icdCode1) >= 390 and float(icdCode1) <= 459:
        validIndices.append(i)
    i+=1

dfDiagnosisFiltered = dfDiagnosis.iloc[validIndices]
filteredPatientIds = set(dfDiagnosisFiltered['patientunitstayid'].tolist())

3351763 in filteredPatientIds

False

In [104]:
dfDiagnosis = pd.read_csv('diagnosis.csv', na_values="nan")
dfDiagnosis = dfDiagnosis.dropna()

In [105]:
dfPatient = pd.read_csv('patient.csv')

In [106]:
dfPatient

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141764,129391,Female,87,Caucasian,59,91,NaN,157.5,23:36:00,...,ICU to SDU,2,stepdown/other,NaN,NaN,18:58:00,344,Home,Alive,002-1039
1,141765,129391,Female,87,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",157.5,23:36:00,...,Emergency Department,1,admit,46.5,45.0,13:14:00,2250,Step-Down Unit (SDU),Alive,002-1039
2,143870,131022,Male,76,Caucasian,68,103,"Endarterectomy, carotid",167.0,20:46:00,...,Operating Room,1,admit,77.5,79.4,10:00:00,793,Floor,Alive,002-12289
3,144815,131736,Female,34,Caucasian,56,82,"Overdose, other toxin, poison or drug",172.7,01:44:00,...,Emergency Department,1,admit,60.3,60.7,20:48:00,1121,Other External,Alive,002-1116
4,145427,132209,Male,61,Caucasian,68,103,"GI perforation/rupture, surgery for",177.8,23:48:00,...,Operating Room,1,admit,91.7,93.1,22:47:00,1369,Floor,Alive,002-12243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515,3351763,2741766,Female,62,Caucasian,459,1108,"Fistula/abscess, surgery for (not inflammatory...",165.1,16:08:00,...,Operating Room,1,admit,134.5,133.3,19:24:00,5394,Step-Down Unit (SDU),Alive,035-10391
2516,3352230,2742186,Male,41,African American,458,1107,"CABG alone, coronary artery bypass grafting",177.8,21:21:00,...,Operating Room,2,transfer,127.0,128.5,21:34:00,4261,Telemetry,Alive,035-10089
2517,3352231,2742186,Male,41,African American,458,1104,"Infarction, acute myocardial (MI)",177.8,21:21:00,...,Direct Admit,1,admit,127.0,135.2,22:26:00,1369,Other ICU,Alive,035-10089
2518,3352333,2742269,Male,72,Caucasian,458,1111,GI obstruction,177.8,20:00:00,...,Other Hospital,1,admit,68.3,66.5,17:26:00,4166,Floor,Alive,035-10041


In [107]:
dfPatientNew = dfPatient[['patientunitstayid', 'gender', 'age', 'admissionweight', 'admissionheight', 'ethnicity']]
dfPatientNew

,patientunitstayid,gender,age,admissionweight,admissionheight,ethnicity
0,141764,Female,87,NaN,157.5,Caucasian
1,141765,Female,87,46.5,157.5,Caucasian
2,143870,Male,76,77.5,167.0,Caucasian
3,144815,Female,34,60.3,172.7,Caucasian
4,145427,Male,61,91.7,177.8,Caucasian
...,...,...,...,...,...,...
2515,3351763,Female,62,134.5,165.1,Caucasian
2516,3352230,Male,41,127.0,177.8,African American
2517,3352231,Male,41,127.0,177.8,African American
2518,3352333,Male,72,68.3,177.8,Caucasian


In [108]:
dfVitalAperiodic = pd.read_csv('vitalAperiodic.csv')
dfVitalAperiodic

,vitalaperiodicid,patientunitstayid,observationoffset,noninvasivesystolic,noninvasivediastolic,noninvasivemean,paop,cardiacoutput,cardiacinput,svr,svri,pvr,pvri
0,3661418,141764,81,171.0,90.0,116.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3661424,141764,334,153.0,78.0,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3661417,141764,77,176.0,87.0,107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3661419,141764,165,173.0,106.0,128.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3661421,141764,255,182.0,103.0,133.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
274083,445476282,3353113,2206,146.0,97.0,116.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274084,445476261,3353113,1269,121.0,87.0,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274085,445476252,3353113,710,127.0,89.0,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274086,445476260,3353113,1257,123.0,82.0,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
dfVitalAperiodic = dfVitalAperiodic.groupby(['patientunitstayid'], as_index=False).mean()[['patientunitstayid','vitalaperiodicid', 'observationoffset','noninvasivesystolic',
                                                                                           'noninvasivediastolic','noninvasivemean']]
dfVitalAperiodic

,patientunitstayid,vitalaperiodicid,observationoffset,noninvasivesystolic,noninvasivediastolic,noninvasivemean
0,141764,3661420.5,204.125000,172.750000,93.375000,125.625000
1,141765,5600897.5,961.900000,146.033333,72.800000,98.400000
2,143870,5022024.5,296.500000,102.500000,53.166667,74.500000
3,144815,1196461.5,480.333333,114.388889,72.777778,88.777778
4,145427,2150246.5,358.250000,116.666667,62.833333,83.208333
...,...,...,...,...,...,...
2326,3351763,443446038.5,2876.161765,135.985294,63.323529,89.279412
2327,3352230,443136851.0,2360.507246,117.101449,65.260870,84.304348
2328,3352231,445040552.0,655.600000,128.000000,71.800000,91.800000
2329,3352333,442861118.5,1505.014286,146.600000,81.628571,106.228571


In [110]:
dfPatientNew = dfPatientNew.merge(dfVitalAperiodic, how='left', left_on='patientunitstayid', right_on='patientunitstayid')
dfPatientNew

,patientunitstayid,gender,age,admissionweight,admissionheight,ethnicity,vitalaperiodicid,observationoffset,noninvasivesystolic,noninvasivediastolic,noninvasivemean
0,141764,Female,87,NaN,157.5,Caucasian,3661420.5,204.125000,172.750000,93.375000,125.625000
1,141765,Female,87,46.5,157.5,Caucasian,5600897.5,961.900000,146.033333,72.800000,98.400000
2,143870,Male,76,77.5,167.0,Caucasian,5022024.5,296.500000,102.500000,53.166667,74.500000
3,144815,Female,34,60.3,172.7,Caucasian,1196461.5,480.333333,114.388889,72.777778,88.777778
4,145427,Male,61,91.7,177.8,Caucasian,2150246.5,358.250000,116.666667,62.833333,83.208333
...,...,...,...,...,...,...,...,...,...,...,...
2515,3351763,Female,62,134.5,165.1,Caucasian,443446038.5,2876.161765,135.985294,63.323529,89.279412
2516,3352230,Male,41,127.0,177.8,African American,443136851.0,2360.507246,117.101449,65.260870,84.304348
2517,3352231,Male,41,127.0,177.8,African American,445040552.0,655.600000,128.000000,71.800000,91.800000
2518,3352333,Male,72,68.3,177.8,Caucasian,442861118.5,1505.014286,146.600000,81.628571,106.228571


In [111]:
dfPatientNew['cardioDiagnosis'] = dfPatientNew['patientunitstayid'].apply(lambda x : x in filteredPatientIds)
dfPatientNew

,patientunitstayid,gender,age,admissionweight,admissionheight,ethnicity,vitalaperiodicid,observationoffset,noninvasivesystolic,noninvasivediastolic,noninvasivemean,cardioDiagnosis
1,141765,Female,87,46.5,157.5,Caucasian,5600897.5,961.900000,146.033333,72.800000,98.400000,False
2,143870,Male,76,77.5,167.0,Caucasian,5022024.5,296.500000,102.500000,53.166667,74.500000,False
3,144815,Female,34,60.3,172.7,Caucasian,1196461.5,480.333333,114.388889,72.777778,88.777778,False
4,145427,Male,61,91.7,177.8,Caucasian,2150246.5,358.250000,116.666667,62.833333,83.208333,False
6,147307,Female,55,72.5,157.5,Caucasian,6918464.5,241.562500,128.375000,65.250000,89.062500,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2515,3351763,Female,62,134.5,165.1,Caucasian,443446038.5,2876.161765,135.985294,63.323529,89.279412,False
2516,3352230,Male,41,127.0,177.8,African American,443136851.0,2360.507246,117.101449,65.260870,84.304348,True
2517,3352231,Male,41,127.0,177.8,African American,445040552.0,655.600000,128.000000,71.800000,91.800000,True
2518,3352333,Male,72,68.3,177.8,Caucasian,442861118.5,1505.014286,146.600000,81.628571,106.228571,False


In [112]:
dfTreatment = pd.read_csv('treatment.csv')[['patientunitstayid','treatmentstring']]
dfTreatment 

,patientunitstayid,treatmentstring
0,242895,cardiovascular|arrhythmias|anticoagulant admin...
1,242895,cardiovascular|consultations|Cardiology consul...
2,242895,cardiovascular|non-operative procedures|extern...
3,242895,pulmonary|vascular disorders|VTE prophylaxis|l...
4,242895,cardiovascular|consultations|Cardiology consul...
...,...,...
38285,3352230,pulmonary|ventilation and oxygenation|mechanic...
38286,3352230,neurologic|pain / agitation / altered mentatio...
38287,3352231,cardiovascular|non-operative procedures|intraa...
38288,3352333,gastrointestinal|consultations|Gastroenterolog...


In [113]:
dfTreatment = dfTreatment.groupby(['patientunitstayid'], as_index=False).agg({'treatmentstring': ' '.join})
dfTreatment['treated'] = np.where(dfTreatment['treatmentstring'].str.contains("cardiovascular"), 'True', 'False')
dfTreatment = dfTreatment[['patientunitstayid', 'treated']]
dfTreatment

,patientunitstayid,treated
0,242895,True
1,243097,True
2,243999,True
3,244477,True
4,244707,True
...,...,...
1905,3351763,True
1906,3352230,True
1907,3352231,True
1908,3352333,False


In [115]:
dfPatientNew = dfPatientNew.merge(dfTreatment, how='left', left_on='patientunitstayid', right_on='patientunitstayid')
dfPatientNew = dfPatientNew.dropna()
dfPatientNew

,patientunitstayid,gender,age,admissionweight,admissionheight,ethnicity,vitalaperiodicid,observationoffset,noninvasivesystolic,noninvasivediastolic,noninvasivemean,cardioDiagnosis,treated_x,treated_y
108,242895,Male,80,101.2,170.18,Caucasian,15957733.0,1102.800000,136.371429,73.028571,87.857143,True,True,True
109,243097,Male,49,127.0,185.40,Caucasian,17382422.5,1191.789474,128.947368,77.815789,88.815789,True,True,True
110,243999,Female,70,98.4,157.50,Caucasian,11676875.0,806.041096,101.589041,55.520548,65.945205,True,True,True
111,244477,Female,55,113.7,165.10,Caucasian,16120409.0,197.000000,112.142857,60.285714,72.285714,False,True,True
112,246997,Male,62,50.1,172.70,Caucasian,18137589.0,733.131148,96.245902,66.819672,79.147541,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133,3351763,Female,62,134.5,165.10,Caucasian,443446038.5,2876.161765,135.985294,63.323529,89.279412,False,True,True
2134,3352230,Male,41,127.0,177.80,African American,443136851.0,2360.507246,117.101449,65.260870,84.304348,True,True,True
2135,3352231,Male,41,127.0,177.80,African American,445040552.0,655.600000,128.000000,71.800000,91.800000,True,True,True
2136,3352333,Male,72,68.3,177.80,Caucasian,442861118.5,1505.014286,146.600000,81.628571,106.228571,False,False,False


In [ ]:
dfPatientNew.to_csv('', index=False)